In [ ]:
# Importing functions and classes we'll use

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from warnings import catch_warnings
from warnings import filterwarnings
from tqdm import tqdm
import keras
import sys
import scipy.stats
import json
import numpy.fft
import time
from decimal import Decimal
import math
import seaborn as sns
%matplotlib inline

In [ ]:
# fix random seed for reproducibility
tf.random.set_seed(1234)

In [ ]:
df = pd.read_excel('/content/sample_data/new_dataset.xlsx')
node1_delay = df[['node1_delay']]

dataset = node1_delay.values

# MA Filter Implementation

In [ ]:
# ASAP
class Metrics(object):
    def __init__(self, values):
        self.set_values( values )

    def set_values(self, values):
        self.values = values
        self.r = self.k = None

    @property
    def kurtosis(self):
        if self.k is None:
            self.k = scipy.stats.kurtosis(self.values)
        return self.k

    @property
    def roughness(self):
        if self.r is None:
            self.r = np.std(np.diff(self.values))
        return self.r

class ACF(Metrics):
    CORR_THRESH = 0.2
    def __init__(self, values, max_lag=None):
        super(ACF, self).__init__(values)
        if max_lag is None:
            max_lag = len(values) / 5
        self.max_lag = int(max_lag)
        self.max_acf = 0.0

        # Calculate autocorrelation via FFT
        # Demean
        demeaned = values - np.mean(values)
        # Pad data to power of 2
        l = int(2.0 ** (int(math.log(len(demeaned),2.0)) + 1))
        padded = np.append(demeaned, ([0.0] * (l - len(demeaned))))
        # FFT and inverse FFT
        F_f = numpy.fft.fft( padded )
        R_t = numpy.fft.ifft( F_f * np.conjugate(F_f) )
        self.correlations = R_t[:int(max_lag)].real / R_t[0].real

        # Find autocorrelation peaks
        self.peaks = []
        if len(self.correlations) >1 :
            positive = self.correlations[1] > self.correlations[0]
            max = 1
            for i in range(2, len(self.correlations)):
                if not positive and self.correlations[i] > self.correlations[i-1]:
                    max = i
                    positive = not positive
                elif positive and self.correlations[i] > self.correlations[max]:
                    max = i
                elif positive and self.correlations[i] < self.correlations[i-1]:
                    if max > 1 and self.correlations[max] > self.CORR_THRESH:
                        self.peaks.append(max)
                        if self.correlations[max] > self.max_acf:
                            self.max_acf = self.correlations[max]
                    positive = not positive
        # If there is no autocorrelation peak within the MAX_WINDOW boundary,
        # try windows from the largest to the smallest
        if len(self.peaks) <= 1:
            self.peaks = range(2, len(self.correlations))

def moving_average(data, _range):
    ret = np.cumsum(data)
    ret[_range:] = ret[int(_range):] - ret[:-int(_range)]
    return ret[int(_range) - 1:] / _range

def SMA(data, _range, slide):
    ret = moving_average(data, int(_range))[::int(slide)]
    return list(ret)

def binary_search(head,tail,data,min_obj,orig_kurt,window_size):
    while head <= tail:
        w = int(round((head + tail) / 2.0))
        smoothed = SMA(data,w,1)
        metrics  = Metrics(smoothed)
        if metrics.kurtosis >= orig_kurt:
            if metrics.roughness < min_obj:
                window_size = w
                min_obj = metrics.roughness
            head = w + 1
        else:
            tail = w - 1
    return window_size

def smooth_ASAP(data, max_window=5, resolution=None):
    data = np.array(data)
    # Preaggregate according to resolution
    slide_size = 1
    window_size = 1
    if resolution and len(data) >= 2 * resolution:
        slide_size = len(data) / resolution
        data = SMA(data, slide_size, slide_size)
    acf         = ACF(data, max_lag=len(data) / max_window)
    peaks       = acf.peaks
    orig_kurt   = acf.kurtosis
    min_obj     = acf.roughness
    lb          = 1
    largest_feasible = -1
    tail = len(data) / max_window
    for i in range(len(peaks) - 1, -1, -1):
        w = peaks[i]

        if w < lb or w == 1:
            break
        elif math.sqrt(1 - acf.correlations[w]) * window_size > math.sqrt(1 - acf.correlations[window_size]) * w:
            continue

        smoothed = SMA(data, w, 1)
        metrics = Metrics(smoothed)
        if metrics.roughness < min_obj and metrics.kurtosis >= orig_kurt:
            min_obj = metrics.roughness
            window_size = w
            lb = round( max(w*math.sqrt( (acf.max_acf -1) / (acf.correlations[w]-1) ), lb) )
    if largest_feasible > 0:
        if largest_feasible < len(peaks) - 2:
            tail = peaks[largest_feasible + 1]
        lb = max(lb, peaks[largest_feasible] + 1)

    window_size = binary_search(lb, tail, data, min_obj, orig_kurt, window_size)
    return window_size, slide_size

# Algorithm to transfer data in tabular format

In [ ]:
def create_multistep_dataset(data, n_input, n_out=1):
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end]
            X.append(x_input)
            y.append(data[in_end:out_end])
        # move along one time step
        in_start += 1
    return np.array(X), np.array(y)

# ESN Model Library

In [ ]:
def correct_dimensions(s, targetlength):
    """checks the dimensionality of some numeric argument s, broadcasts it
       to the specified length if possible.

    Args:
        s: None, scalar or 1D array
        targetlength: expected length of s

    Returns:
        None if s is None, else numpy vector of length targetlength
    """
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s


def identity(x):
    return x


class ESN():

    def __init__(self, n_inputs, n_outputs, n_reservoir=200,
                 spectral_radius=0.95, sparsity=0, noise=0.001, input_shift=None,
                 input_scaling=None, teacher_forcing=True, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 out_activation=identity, inverse_out_activation=identity,
                 random_state=None, silent=True):
        """
        Args:
            n_inputs: nr of input dimensions
            n_outputs: nr of output dimensions
            n_reservoir: nr of reservoir neurons
            spectral_radius: spectral radius of the recurrent weight matrix
            sparsity: proportion of recurrent weights set to zero
            noise: noise added to each neuron (regularization)
            input_shift: scalar or vector of length n_inputs to add to each
                        input dimension before feeding it to the network.
            input_scaling: scalar or vector of length n_inputs to multiply
                        with each input dimension before feeding it to the netw.
            teacher_forcing: if True, feed the target back into output units
            teacher_scaling: factor applied to the target signal
            teacher_shift: additive term applied to the target signal
            out_activation: output activation function (applied to the readout)
            inverse_out_activation: inverse of the output activation function
            random_state: positive integer seed, np.rand.RandomState object,
                          or None to use numpy's builting RandomState.
            silent: supress messages
        """
        # check for proper dimensionality of all arguments and write them down.
        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.n_outputs = n_outputs
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.input_shift = correct_dimensions(input_shift, n_inputs)
        self.input_scaling = correct_dimensions(input_scaling, n_inputs)

        self.teacher_scaling = teacher_scaling
        self.teacher_shift = teacher_shift

        self.out_activation = out_activation
        self.inverse_out_activation = inverse_out_activation
        self.random_state = random_state

        # the given random_state might be either an actual RandomState object,
        # a seed or None (in which case we use numpy's builtin RandomState)
        if isinstance(random_state, np.random.RandomState):
            self.random_state_ = random_state
        elif random_state:
            try:
                self.random_state_ = np.random.RandomState(random_state)
            except TypeError as e:
                raise Exception("Invalid seed: " + str(e))
        else:
            self.random_state_ = np.random.mtrand._rand

        self.teacher_forcing = teacher_forcing
        self.silent = silent
        self.initweights()

    def initweights(self):
        # initialize recurrent weights:
        # begin with a random matrix centered around zero:
        W = self.random_state_.rand(self.n_reservoir, self.n_reservoir) - 0.5
        # delete the fraction of connections given by (self.sparsity):
        W[self.random_state_.rand(*W.shape) < self.sparsity] = 0
        # compute the spectral radius of these weights:
        radius = np.max(np.abs(np.linalg.eigvals(W)))
        # rescale them to reach the requested spectral radius:
        self.W = W * (self.spectral_radius / radius)

        # random input weights:
        self.W_in = self.random_state_.rand(
            self.n_reservoir, self.n_inputs) * 2 - 1
        # random feedback (teacher forcing) weights:
        self.W_feedb = self.random_state_.rand(
            self.n_reservoir, self.n_outputs) * 2 - 1

    def _update(self, state, input_pattern, output_pattern):
        """performs one update step.

        i.e., computes the next network state by applying the recurrent weights
        to the last state & and feeding in the current input and output patterns
        """
        if self.teacher_forcing:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern)
                             + np.dot(self.W_feedb, output_pattern))
        else:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern))
        return (np.tanh(preactivation)
                + self.noise * (self.random_state_.rand(self.n_reservoir) - 0.5))

    def _scale_inputs(self, inputs):
        """for each input dimension j: multiplies by the j'th entry in the
        input_scaling argument, then adds the j'th entry of the input_shift
        argument."""
        if self.input_scaling is not None:
            inputs = np.dot(inputs, np.diag(self.input_scaling))
        if self.input_shift is not None:
            inputs = inputs + self.input_shift
        return inputs

    def _scale_teacher(self, teacher):
        """multiplies the teacher/target signal by the teacher_scaling argument,
        then adds the teacher_shift argument to it."""
        if self.teacher_scaling is not None:
            teacher = teacher * self.teacher_scaling
        if self.teacher_shift is not None:
            teacher = teacher + self.teacher_shift
        return teacher

    def _unscale_teacher(self, teacher_scaled):
        """inverse operation of the _scale_teacher method."""
        if self.teacher_shift is not None:
            teacher_scaled = teacher_scaled - self.teacher_shift
        if self.teacher_scaling is not None:
            teacher_scaled = teacher_scaled / self.teacher_scaling
        return teacher_scaled

    def fit(self, inputs, outputs, inspect=False):
        """
        Collect the network's reaction to training data, train readout weights.

        Args:
            inputs: array of dimensions (N_training_samples x n_inputs)
            outputs: array of dimension (N_training_samples x n_outputs)
            inspect: show a visualisation of the collected reservoir states

        Returns:
            the network's output on the training data, using the trained weights
        """
        # transform any vectors of shape (x,) into vectors of shape (x,1):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        if outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))
        # transform input and teacher signal:
        inputs_scaled = self._scale_inputs(inputs)
        teachers_scaled = self._scale_teacher(outputs)

        if not self.silent:
            print("harvesting states...")
        # step the reservoir through the given input,output pairs:
        states = np.zeros((inputs.shape[0], self.n_reservoir))
        for n in range(1, inputs.shape[0]):
            states[n, :] = self._update(states[n - 1], inputs_scaled[n, :],
                                        teachers_scaled[n - 1, :])

        # learn the weights, i.e. find the linear combination of collected
        # network states that is closest to the target output
        if not self.silent:
            print("fitting...")
        # we'll disregard the first few states:
        transient = min(int(inputs.shape[1] / 10), 100)
        # include the raw inputs:
        extended_states = np.hstack((states, inputs_scaled))
        # Solve for W_out:
        self.W_out = np.dot(np.linalg.pinv(extended_states[transient:, :]),
                            self.inverse_out_activation(teachers_scaled[transient:, :])).T

        # remember the last state for later:
        self.laststate = states[-1, :]
        self.lastinput = inputs[-1, :]
        self.lastoutput = teachers_scaled[-1, :]

        # optionally visualize the collected states
        if inspect:
            from matplotlib import pyplot as plt
            # (^-- we depend on matplotlib only if this option is used)
            plt.figure(
                figsize=(states.shape[0] * 0.0025, states.shape[1] * 0.01))
            plt.imshow(extended_states.T, aspect='auto',
                       interpolation='nearest')
            plt.colorbar()

        if not self.silent:
            print("training error:")
        # apply learned weights to the collected states:
        pred_train = self._unscale_teacher(self.out_activation(
            np.dot(extended_states, self.W_out.T)))
        if not self.silent:
            print(np.sqrt(np.mean((pred_train - outputs)**2)))
        return pred_train

    def predict(self, inputs, continuation=True):
        """
        Apply the learned weights to the network's reactions to new input.

        Args:
            inputs: array of dimensions (N_test_samples x n_inputs)
            continuation: if True, start the network from the last training state

        Returns:
            Array of output activations
        """
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        n_samples = inputs.shape[0]

        if continuation:
            laststate = self.laststate
            lastinput = self.lastinput
            lastoutput = self.lastoutput
        else:
            laststate = np.zeros(self.n_reservoir)
            lastinput = np.zeros(self.n_inputs)
            lastoutput = np.zeros(self.n_outputs)

        inputs = np.vstack([lastinput, self._scale_inputs(inputs)])
        states = np.vstack(
            [laststate, np.zeros((n_samples, self.n_reservoir))])
        outputs = np.vstack(
            [lastoutput, np.zeros((n_samples, self.n_outputs))])

        for n in range(n_samples):
            states[
                n + 1, :] = self._update(states[n, :], inputs[n + 1, :], outputs[n, :])
            outputs[n + 1, :] = self.out_activation(np.dot(self.W_out,
                                                           np.concatenate([states[n + 1, :], inputs[n + 1, :]])))

        return self._unscale_teacher(self.out_activation(outputs[1:]))

# MIMO Strategy

#### LSTM Model

##### Finding the best hyper-parameters

In [ ]:
!pip install -q optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.4 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
horizon = 2

# Hyper Parameters tuning

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

def objective(trial):
    # define search space for hyperparameters
    look_back = trial.suggest_int('look_back', 5, 300)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 5, 300)
    epochs = trial.suggest_int('epochs', 10, 100)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
    print(trainX.shape)

    # Crée et entraîne le modèle pour l'horizon de prévision i
    model = Sequential()
    for i in range(num_hidden_layers):
        num_units = trial.suggest_int(f'lstm_units_layer_{i}', 8, 256, log=True)
        return_sequences = (i < num_hidden_layers - 1)
        model.add(LSTM(units=num_units, return_sequences=return_sequences))
    model.add(Dense(horizon))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0)
    validPredict = model.predict(validX, verbose=0)

    # calculate root mean squared error
    validScore = np.sqrt(mean_squared_error(validY, validPredict))

    return validScore

study = optuna.create_study(direction='minimize', sampler=TPESampler())

# Record the starting time to generate predictions
start_time = time.time()

study.optimize(objective, n_trials=50, n_jobs=-1)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("LSTM HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

best_params = study.best_params
best_error = study.best_value

print("Done")
print(f"best_params: {best_params}, best_error: {best_error}")

[I 2023-09-05 07:49:25,490] A new study created in memory with name: no-name-fbe15e96-8df8-413d-9ade-f6191159dabb


Window Size:  10
17991 16192 1799 2000
(17958, 1, 32)
(17917, 1, 73)


<ipython-input-9-f3b7a5822bef>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
[I 2023-09-05 07:51:49,713] Trial 1 finished with value: 0.023953876628435365 and parameters: {'look_back': 32, 'num_hidden_layers': 5, 'learning_rate': 0.0002486158494589482, 'batch_size': 123, 'epochs': 63, 'lstm_units_layer_0': 8, 'lstm_units_layer_1': 84, 'lstm_units_layer_2': 16, 'lstm_units_layer_3': 162, 'lstm_units_layer_4': 29}. Best is trial 1 with value: 0.023953876628435365.
<ipython-input-9-f3b7a5822bef>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featur

(17942, 1, 48)


[I 2023-09-05 07:55:35,427] Trial 0 finished with value: 0.05171286313859949 and parameters: {'look_back': 73, 'num_hidden_layers': 9, 'learning_rate': 0.04175647503853579, 'batch_size': 123, 'epochs': 95, 'lstm_units_layer_0': 202, 'lstm_units_layer_1': 242, 'lstm_units_layer_2': 56, 'lstm_units_layer_3': 33, 'lstm_units_layer_4': 57, 'lstm_units_layer_5': 216, 'lstm_units_layer_6': 59, 'lstm_units_layer_7': 58, 'lstm_units_layer_8': 41}. Best is trial 1 with value: 0.023953876628435365.
<ipython-input-9-f3b7a5822bef>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

(17956, 1, 34)


[I 2023-09-05 10:27:38,861] Trial 48 finished with value: 0.021720396441367702 and parameters: {'look_back': 34, 'num_hidden_layers': 1, 'learning_rate': 0.0016806971889211637, 'batch_size': 23, 'epochs': 99, 'lstm_units_layer_0': 153}. Best is trial 41 with value: 0.020879777783223082.
<ipython-input-9-f3b7a5822bef>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


(17933, 1, 57)


[I 2023-09-05 10:32:35,691] Trial 49 finished with value: 0.02111907703570876 and parameters: {'look_back': 57, 'num_hidden_layers': 3, 'learning_rate': 0.0009834583346548876, 'batch_size': 49, 'epochs': 86, 'lstm_units_layer_0': 149, 'lstm_units_layer_1': 174, 'lstm_units_layer_2': 41}. Best is trial 41 with value: 0.020879777783223082.
[I 2023-09-05 10:36:04,854] Trial 47 finished with value: 0.025932435893827148 and parameters: {'look_back': 63, 'num_hidden_layers': 2, 'learning_rate': 0.0013656938055084647, 'batch_size': 6, 'epochs': 99, 'lstm_units_layer_0': 153, 'lstm_units_layer_1': 181}. Best is trial 41 with value: 0.020879777783223082.


done
LSTM HyperParameters Tuning Elapsed Time : 9999.36385 seconds
Done
best_params: {'look_back': 51, 'num_hidden_layers': 2, 'learning_rate': 0.001519335526892746, 'batch_size': 51, 'epochs': 82, 'lstm_units_layer_0': 96, 'lstm_units_layer_1': 208}, best_error: 0.020879777783223082


In [ ]:
horizon = 4

# Hyper Parameters tuning

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

def objective(trial):
    # define search space for hyperparameters
    look_back = trial.suggest_int('look_back', 5, 300)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 5, 300)
    epochs = trial.suggest_int('epochs', 10, 100)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
    print(trainX.shape)

    # Crée et entraîne le modèle pour l'horizon de prévision i
    model = Sequential()
    for i in range(num_hidden_layers):
        num_units = trial.suggest_int(f'lstm_units_layer_{i}', 8, 256, log=True)
        return_sequences = (i < num_hidden_layers - 1)
        model.add(LSTM(units=num_units, return_sequences=return_sequences))
    model.add(Dense(horizon))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0)
    validPredict = model.predict(validX, verbose=0)

    # calculate root mean squared error
    validScore = np.sqrt(mean_squared_error(validY, validPredict))

    return validScore

study = optuna.create_study(direction='minimize', sampler=TPESampler())

# Record the starting time to generate predictions
start_time = time.time()

study.optimize(objective, n_trials=50, n_jobs=-1)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("LSTM HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

best_params = study.best_params
best_error = study.best_value

print("Done")
print(f"best_params: {best_params}, best_error: {best_error}")

[I 2023-09-05 10:36:04,897] A new study created in memory with name: no-name-76106382-f830-4891-bbfd-4c5c6efb0a5a
<ipython-input-10-3656848860d2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Window Size:  10
17991 16192 1799 2000
(17770, 1, 218)(17959, 1, 29)



/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
[I 2023-09-05 10:39:15,479] Trial 0 finished with value: 0.036401587641882976 and parameters: {'look_back': 29, 'num_hidden_layers': 7, 'learning_rate': 2.0431540891277735e-05, 'batch_size': 109, 'epochs': 57, 'lstm_units_layer_0': 62, 'lstm_units_layer_1': 26, 'lstm_units_layer_2': 11, 'lstm_units_layer_3': 127, 'lstm_units_layer_4': 142, 'lstm_units_layer_5': 82, 'lstm_units_layer_6': 69}. Best is trial 0 with value: 0.036401587641882976.
<ipython-input-10-3656848860d2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizer

(17710, 1, 278)


[I 2023-09-05 10:41:06,425] Trial 1 finished with value: 0.03304651642641758 and parameters: {'look_back': 218, 'num_hidden_layers': 10, 'learning_rate': 0.00013597717429592702, 'batch_size': 15, 'epochs': 10, 'lstm_units_layer_0': 12, 'lstm_units_layer_1': 9, 'lstm_units_layer_2': 14, 'lstm_units_layer_3': 13, 'lstm_units_layer_4': 179, 'lstm_units_layer_5': 113, 'lstm_units_layer_6': 11, 'lstm_units_layer_7': 147, 'lstm_units_layer_8': 48, 'lstm_units_layer_9': 40}. Best is trial 1 with value: 0.03304651642641758.
<ipython-input-10-3656848860d2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__i

...

[I 2023-09-05 11:16:52,176] Trial 27 finished with value: 0.02909870248957507 and parameters: {'look_back': 60, 'num_hidden_layers': 3, 'learning_rate': 0.03383266226383165, 'batch_size': 202, 'epochs': 51, 'lstm_units_layer_0': 34, 'lstm_units_layer_1': 78, 'lstm_units_layer_2': 48}. Best is trial 26 with value: 0.02686025211281613.
<ipython-input-10-3656848860d2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


(17955, 1, 33)


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
[I 2023-09-05 11:17:48,398] Trial 28 finished with value: 0.05241592987372683 and parameters: {'look_back': 55, 'num_hidden_layers': 3, 'learning_rate': 0.0912349521660902, 'batch_size': 193, 'epochs': 55, 'lstm_units_layer_0': 31, 'lstm_units_layer_1': 68, 'lstm_units_layer_2': 57}. Best is trial 26 with value: 0.02686025211281613.
<ipython-input-10-3656848860d2>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


(17861, 1, 127)


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
horizon = 6

# Hyper Parameters tuning

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

def objective(trial):
    # define search space for hyperparameters
    look_back = trial.suggest_int('look_back', 5, 300)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 5, 300)
    epochs = trial.suggest_int('epochs', 10, 100)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
    print(trainX.shape)

    # Crée et entraîne le modèle pour l'horizon de prévision i
    model = Sequential()
    for i in range(num_hidden_layers):
        num_units = trial.suggest_int(f'lstm_units_layer_{i}', 8, 256, log=True)
        return_sequences = (i < num_hidden_layers - 1)
        model.add(LSTM(units=num_units, return_sequences=return_sequences))
    model.add(Dense(horizon))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0)
    validPredict = model.predict(validX, verbose=0)

    # calculate root mean squared error
    validScore = np.sqrt(mean_squared_error(validY, validPredict))

    return validScore

study = optuna.create_study(direction='minimize', sampler=TPESampler())

# Record the starting time to generate predictions
start_time = time.time()

study.optimize(objective, n_trials=30, n_jobs=-1)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("LSTM HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

best_params = study.best_params
best_error = study.best_value

print("Done")
print(f"best_params: {best_params}, best_error: {best_error}")

[I 2023-09-05 12:22:41,542] A new study created in memory with name: no-name-d052f972-fc21-4b9c-87eb-05b08c061fdb


Window Size:  10
17991 16192 1799 2000
(17741, 1, 245)
(17754, 1, 232)


<ipython-input-9-51f619b15016>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
[I 2023-09-05 12:23:21,457] Trial 0 finished with value: 0.04374347727548258 and parameters: {'look_back': 245, 'num_hidden_layers': 1, 'learning_rate': 9.600928263181503e-05, 'batch_size': 278, 'epochs': 74, 'lstm_units_layer_0': 22}. Best is trial 0 with value: 0.04374347727548258.
<ipython-input-9-51f619b15016>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(

(17960, 1, 26)


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
[I 2023-09-05 12:23:35,366] Trial 1 finished with value: 0.04641520316491024 and parameters: {'look_back': 232, 'num_hidden_layers': 2, 'learning_rate': 7.087517373904725e-05, 'batch_size': 207, 'epochs': 24, 'lstm_units_layer_0': 169, 'lstm_units_layer_1': 38}. Best is trial 0 with value: 0.04374347727548258.
<ipython-input-9-51f619b15016>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

(17844, 1, 142)


[I 2023-09-05 13:49:52,455] Trial 27 finished with value: 0.039891745896084845 and parameters: {'look_back': 113, 'num_hidden_layers': 8, 'learning_rate': 4.179702941733285e-05, 'batch_size': 155, 'epochs': 84, 'lstm_units_layer_0': 53, 'lstm_units_layer_1': 24, 'lstm_units_layer_2': 58, 'lstm_units_layer_3': 100, 'lstm_units_layer_4': 9, 'lstm_units_layer_5': 30, 'lstm_units_layer_6': 16, 'lstm_units_layer_7': 8}. Best is trial 7 with value: 0.03235792738580053.
<ipython-input-9-51f619b15016>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


(17839, 1, 147)


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
[I 2023-09-05 13:52:06,989] Trial 29 finished with value: 0.03660628651507756 and parameters: {'look_back': 147, 'num_hidden_layers': 3, 'learning_rate': 0.00037981574617246614, 'batch_size': 52, 'epochs': 48, 'lstm_units_layer_0': 85, 'lstm_units_layer_1': 11, 'lstm_units_layer_2': 91}. Best is trial 7 with value: 0.03235792738580053.
[I 2023-09-05 13:52:14,073] Trial 28 finished with value: 0.05184725187415518 and parameters: {'look_back': 142, 'num_hidden_layers': 8, 'learning_rate': 4.6509611864338884e-05, 'batch_size': 142, 'epochs': 48, 'lstm_units_layer_0': 94, 'lstm_units_layer_1': 23, 'lstm_units_layer_2': 63, 'lstm_units_layer_3': 117, 'lstm_units_layer_4': 32, 'lstm_units_layer_5': 36, 'lstm_units_layer_6': 16, 'lstm_units_layer_7': 8}. Best is trial 7 with value: 0.03235792738580053.


done
LSTM HyperParameters Tuning Elapsed Time : 5372.53182 seconds
Done
best_params: {'look_back': 91, 'num_hidden_layers': 6, 'learning_rate': 0.0005124476844228014, 'batch_size': 50, 'epochs': 73, 'lstm_units_layer_0': 50, 'lstm_units_layer_1': 28, 'lstm_units_layer_2': 82, 'lstm_units_layer_3': 33, 'lstm_units_layer_4': 22, 'lstm_units_layer_5': 15}, best_error: 0.03235792738580053


In [ ]:
horizon = 8

# Hyper Parameters tuning

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

def objective(trial):
    # define search space for hyperparameters
    look_back = trial.suggest_int('look_back', 5, 300)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 5, 300)
    epochs = trial.suggest_int('epochs', 10, 100)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
    print(trainX.shape)

    # Crée et entraîne le modèle pour l'horizon de prévision i
    model = Sequential()
    for i in range(num_hidden_layers):
        num_units = trial.suggest_int(f'lstm_units_layer_{i}', 8, 256, log=True)
        return_sequences = (i < num_hidden_layers - 1)
        model.add(LSTM(units=num_units, return_sequences=return_sequences))
    model.add(Dense(horizon))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0)
    validPredict = model.predict(validX, verbose=0)

    # calculate root mean squared error
    validScore = np.sqrt(mean_squared_error(validY, validPredict))

    return validScore

study = optuna.create_study(direction='minimize', sampler=TPESampler())

# Record the starting time to generate predictions
start_time = time.time()

study.optimize(objective, n_trials=30, n_jobs=-1)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("LSTM HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

best_params = study.best_params
best_error = study.best_value

print("Done")
print(f"best_params: {best_params}, best_error: {best_error}")

[I 2023-09-05 13:52:14,106] A new study created in memory with name: no-name-a4a20d05-f317-4b92-b047-c37579da0655
<ipython-input-10-27a80ae14fd5>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Window Size:  10
17991 16192 1799 2000
(17964, 1, 20)
(17826, 1, 158)


[I 2023-09-05 13:53:15,349] Trial 1 finished with value: 0.0386272033080371 and parameters: {'look_back': 20, 'num_hidden_layers': 4, 'learning_rate': 0.05522253569453423, 'batch_size': 219, 'epochs': 58, 'lstm_units_layer_0': 25, 'lstm_units_layer_1': 14, 'lstm_units_layer_2': 56, 'lstm_units_layer_3': 17}. Best is trial 1 with value: 0.0386272033080371.
<ipython-input-10-27a80ae14fd5>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


(17739, 1, 245)


[I 2023-09-05 13:53:28,378] Trial 0 finished with value: 0.03907573793513375 and parameters: {'look_back': 158, 'num_hidden_layers': 1, 'learning_rate': 5.018014771015471e-05, 'batch_size': 75, 'epochs': 53, 'lstm_units_layer_0': 140}. Best is trial 1 with value: 0.0386272033080371.
<ipython-input-10-27a80ae14fd5>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

(17864, 1, 120)


[I 2023-09-05 14:44:55,587] Trial 27 finished with value: 0.03794942128953343 and parameters: {'look_back': 129, 'num_hidden_layers': 2, 'learning_rate': 0.00022050666465151084, 'batch_size': 70, 'epochs': 100, 'lstm_units_layer_0': 8, 'lstm_units_layer_1': 94}. Best is trial 15 with value: 0.036570963652281366.
<ipython-input-10-27a80ae14fd5>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


(17892, 1, 92)


[I 2023-09-05 14:50:18,510] Trial 29 finished with value: 0.03920423091814834 and parameters: {'look_back': 92, 'num_hidden_layers': 6, 'learning_rate': 0.005204574620974465, 'batch_size': 58, 'epochs': 65, 'lstm_units_layer_0': 19, 'lstm_units_layer_1': 53, 'lstm_units_layer_2': 142, 'lstm_units_layer_3': 115, 'lstm_units_layer_4': 51, 'lstm_units_layer_5': 8}. Best is trial 15 with value: 0.036570963652281366.
[I 2023-09-05 14:51:22,369] Trial 28 finished with value: 0.03966708093695119 and parameters: {'look_back': 120, 'num_hidden_layers': 6, 'learning_rate': 6.04615387402686e-05, 'batch_size': 62, 'epochs': 99, 'lstm_units_layer_0': 11, 'lstm_units_layer_1': 56, 'lstm_units_layer_2': 154, 'lstm_units_layer_3': 114, 'lstm_units_layer_4': 53, 'lstm_units_layer_5': 8}. Best is trial 15 with value: 0.036570963652281366.


done
LSTM HyperParameters Tuning Elapsed Time : 3548.26525 seconds
Done
best_params: {'look_back': 65, 'num_hidden_layers': 3, 'learning_rate': 0.00019511832310862206, 'batch_size': 8, 'epochs': 66, 'lstm_units_layer_0': 8, 'lstm_units_layer_1': 63, 'lstm_units_layer_2': 240}, best_error: 0.036570963652281366


In [ ]:
horizon = 10

# Hyper Parameters tuning

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

def objective(trial):
    # define search space for hyperparameters
    look_back = trial.suggest_int('look_back', 5, 300)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 5, 300)
    epochs = trial.suggest_int('epochs', 10, 100)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
    print(trainX.shape)

    # Crée et entraîne le modèle pour l'horizon de prévision i
    model = Sequential()
    for i in range(num_hidden_layers):
        num_units = trial.suggest_int(f'lstm_units_layer_{i}', 8, 256, log=True)
        return_sequences = (i < num_hidden_layers - 1)
        model.add(LSTM(units=num_units, return_sequences=return_sequences))
    model.add(Dense(horizon))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0)
    validPredict = model.predict(validX, verbose=0)

    # calculate root mean squared error
    validScore = np.sqrt(mean_squared_error(validY, validPredict))

    return validScore

study = optuna.create_study(direction='minimize', sampler=TPESampler())

# Record the starting time to generate predictions
start_time = time.time()

study.optimize(objective, n_trials=30, n_jobs=-1)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("LSTM HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

best_params = study.best_params
best_error = study.best_value

print("Done")
print(f"best_params: {best_params}, best_error: {best_error}")

[I 2023-09-05 14:51:22,396] A new study created in memory with name: no-name-374783c5-c86e-4449-9c7c-01550d573f12
<ipython-input-11-025fe578aeba>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Window Size:  10
17991 16192 1799 2000
(17948, 1, 34)
(17758, 1, 224)


[I 2023-09-05 14:53:55,114] Trial 1 finished with value: 0.05231549474925235 and parameters: {'look_back': 224, 'num_hidden_layers': 10, 'learning_rate': 0.024089948392788878, 'batch_size': 297, 'epochs': 48, 'lstm_units_layer_0': 159, 'lstm_units_layer_1': 10, 'lstm_units_layer_2': 27, 'lstm_units_layer_3': 58, 'lstm_units_layer_4': 21, 'lstm_units_layer_5': 149, 'lstm_units_layer_6': 20, 'lstm_units_layer_7': 85, 'lstm_units_layer_8': 25, 'lstm_units_layer_9': 41}. Best is trial 1 with value: 0.05231549474925235.
<ipython-input-11-025fe578aeba>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__in

(17881, 1, 101)


[I 2023-09-05 14:55:39,703] Trial 2 finished with value: 0.040439191104931216 and parameters: {'look_back': 101, 'num_hidden_layers': 2, 'learning_rate': 0.00038407156136456767, 'batch_size': 100, 'epochs': 93, 'lstm_units_layer_0': 39, 'lstm_units_layer_1': 76}. Best is trial 2 with value: 0.040439191104931216.
<ipython-input-11-025fe578aeba>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

(17887, 1, 95)


[I 2023-09-05 15:45:01,056] Trial 27 finished with value: 0.0419701187012747 and parameters: {'look_back': 91, 'num_hidden_layers': 3, 'learning_rate': 0.0001214914017461513, 'batch_size': 51, 'epochs': 100, 'lstm_units_layer_0': 34, 'lstm_units_layer_1': 98, 'lstm_units_layer_2': 50}. Best is trial 12 with value: 0.04012357832510543.
<ipython-input-11-025fe578aeba>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


(17831, 1, 151)


[I 2023-09-05 15:53:18,855] Trial 28 finished with value: 0.04106130498361404 and parameters: {'look_back': 95, 'num_hidden_layers': 2, 'learning_rate': 0.0001060791054558371, 'batch_size': 7, 'epochs': 63, 'lstm_units_layer_0': 31, 'lstm_units_layer_1': 95}. Best is trial 12 with value: 0.04012357832510543.
[I 2023-09-05 15:55:35,596] Trial 29 finished with value: 0.04168800493164721 and parameters: {'look_back': 151, 'num_hidden_layers': 2, 'learning_rate': 5.6776166755040616e-05, 'batch_size': 6, 'epochs': 65, 'lstm_units_layer_0': 62, 'lstm_units_layer_1': 20}. Best is trial 12 with value: 0.04012357832510543.


done
LSTM HyperParameters Tuning Elapsed Time : 3853.20098 seconds
Done
best_params: {'look_back': 149, 'num_hidden_layers': 1, 'learning_rate': 0.0024519997428423335, 'batch_size': 83, 'epochs': 99, 'lstm_units_layer_0': 51}, best_error: 0.04012357832510543


In [ ]:
# Best Params
look_back = 52
num_hidden_layers = 1
learning_rate = 0.0025060720938911716
batch_size = 217
epochs = 99
lstm_units_layer_0 = 48

##### Horizon = 2

In [ ]:
# Best Params

look_back = 51
num_hidden_layers = 2
learning_rate = 0.001519335526892746
batch_size = 51
epochs = 82
lstm_units_layer_0 = 96
lstm_units_layer_1 = 208
# best_error: 0.020879777783223082

In [ ]:
horizon = 2

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(LSTM(units=lstm_units_layer_0, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_1))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

# calculate root mean squared error
LSTM_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
LSTM_MAE = mean_absolute_error(testY, testPredict)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("LSTM Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('LSTM Test RMSE : %.5f' % (LSTM_RMSE))
print('LSTM Test MAE : %.5f' % (LSTM_MAE))

Window Size:  10
17991 2000
(17841, 1, 149)
Epoch 1/99


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


215/215 [==============================] - 5s 6ms/step - loss: 0.0039
Epoch 2/99
215/215 [==============================] - 1s 7ms/step - loss: 0.0016
Epoch 3/99
215/215 [==============================] - 1s 6ms/step - loss: 9.7484e-04
Epoch 4/99
215/215 [==============================] - 1s 4ms/step - loss: 8.0636e-04
Epoch 5/99
215/215 [==============================] - 1s 4ms/step - loss: 7.5102e-04
Epoch 6/99
215/215 [==============================] - 1s 4ms/step - loss: 6.9489e-04
Epoch 7/99
215/215 [==============================] - 1s 5ms/step - loss: 7.1108e-04
Epoch 8/99
215/215 [==============================] - 1s 5ms/step - loss: 6.7232e-04
Epoch 9/99
215/215 [==============================] - 1s 4ms/step - loss: 6.5830e-04
Epoch 10/99
215/215 [==============================] - 1s 4ms/step - loss: 6.4791e-04
Epoch 11/99
215/215 [==============================] - 1s 4ms/step - loss: 6.4895e-04
Epoch 12/99
215/215 [==============================] - 1s 4ms/step - loss: 6.1880e

##### Horizon = 4

In [ ]:
# Best Params

# 0.02686025211281613
look_back = 56
num_hidden_layers = 3
learning_rate = 0.0056240033221932485
batch_size = 193
epochs = 78
lstm_units_layer_0 = 37
lstm_units_layer_1 = 82
lstm_units_layer_2 = 49

In [ ]:
horizon = 4

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(LSTM(units=lstm_units_layer_0, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_1, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_2))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
LSTM_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
LSTM_MAE = mean_absolute_error(testY, testPredict)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("LSTM Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('LSTM Test RMSE : %.5f' % (LSTM_RMSE))
print('LSTM Test MAE : %.5f' % (LSTM_MAE))

Window Size:  10
17991 2000
(17932, 1, 56)
Epoch 1/78


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


93/93 [==============================] - 6s 7ms/step - loss: 0.0077
Epoch 2/78
93/93 [==============================] - 1s 8ms/step - loss: 0.0022
Epoch 3/78
93/93 [==============================] - 1s 7ms/step - loss: 0.0018
Epoch 4/78
93/93 [==============================] - 1s 7ms/step - loss: 0.0012
Epoch 5/78
93/93 [==============================] - 1s 13ms/step - loss: 0.0011
Epoch 6/78
93/93 [==============================] - 1s 13ms/step - loss: 9.9254e-04
Epoch 7/78
93/93 [==============================] - 1s 10ms/step - loss: 0.0010
Epoch 8/78
93/93 [==============================] - 1s 14ms/step - loss: 9.2369e-04
Epoch 9/78
93/93 [==============================] - 1s 15ms/step - loss: 8.8778e-04
Epoch 10/78
93/93 [==============================] - 1s 16ms/step - loss: 9.1964e-04
Epoch 11/78
93/93 [==============================] - 1s 7ms/step - loss: 8.8205e-04
Epoch 12/78
93/93 [==============================] - 1s 7ms/step - loss: 8.6774e-04
Epoch 13/78
93/93 [===========

##### Horizon = 6

In [ ]:
# Best Params

look_back = 91
num_hidden_layers = 6
learning_rate = 0.0005124476844228014
batch_size = 50
epochs = 73
lstm_units_layer_0 = 50
lstm_units_layer_1 = 28
lstm_units_layer_2 = 82
lstm_units_layer_3 = 33
lstm_units_layer_4 = 22
lstm_units_layer_5 = 15
# best_error: 0.03235792738580053


In [ ]:
horizon = 6

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(LSTM(units=lstm_units_layer_0, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_1, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_2, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_3, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_4, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_5))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
LSTM_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
LSTM_MAE = mean_absolute_error(testY, testPredict)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("LSTM Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('LSTM Test RMSE : %.5f' % (LSTM_RMSE))
print('LSTM Test MAE : %.5f' % (LSTM_MAE))

Window Size:  10
17991 2000
(17895, 1, 91)
Epoch 1/73


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


358/358 [==============================] - 19s 11ms/step - loss: 0.0204
Epoch 2/73
358/358 [==============================] - 5s 13ms/step - loss: 0.0023
Epoch 3/73
358/358 [==============================] - 4s 11ms/step - loss: 0.0022
Epoch 4/73
358/358 [==============================] - 4s 10ms/step - loss: 0.0017
Epoch 5/73
358/358 [==============================] - 4s 12ms/step - loss: 0.0015
Epoch 6/73
358/358 [==============================] - 4s 12ms/step - loss: 0.0014
Epoch 7/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0014
Epoch 8/73
358/358 [==============================] - 4s 10ms/step - loss: 0.0013
Epoch 9/73
358/358 [==============================] - 5s 13ms/step - loss: 0.0013
Epoch 10/73
358/358 [==============================] - 4s 11ms/step - loss: 0.0013
Epoch 11/73
358/358 [==============================] - 4s 10ms/step - loss: 0.0012
Epoch 12/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0012
Epoch 13/73
358/358 [====

##### Horizon = 8

In [ ]:
# Best Params
look_back = 65
num_hidden_layers = 3
learning_rate = 0.00019511832310862206
batch_size = 8
epochs = 66
lstm_units_layer_0 = 8
lstm_units_layer_1 = 63
lstm_units_layer_2 = 240
# best_error: 0.036570963652281366


In [ ]:
horizon = 8

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(LSTM(units=lstm_units_layer_0, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_1, return_sequences=True))
model.add(LSTM(units=lstm_units_layer_2))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
LSTM_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
LSTM_MAE = mean_absolute_error(testY, testPredict)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("LSTM Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('LSTM Test RMSE : %.5f' % (LSTM_RMSE))
print('LSTM Test MAE : %.5f' % (LSTM_MAE))

Window Size:  10
17991 2000
(17919, 1, 65)
Epoch 1/66


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2240/2240 [==============================] - 17s 5ms/step - loss: 0.0052
Epoch 2/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0022
Epoch 3/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0019
Epoch 4/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0018
Epoch 5/66
2240/2240 [==============================] - 13s 6ms/step - loss: 0.0017
Epoch 6/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0016
Epoch 7/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0016
Epoch 8/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0015
Epoch 9/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0015
Epoch 10/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014
Epoch 11/66
2240/2240 [==============================] - 12s 6ms/step - loss: 0.0014
Epoch 12/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014


##### Horizon = 10

In [ ]:
# Best Params

look_back = 149
num_hidden_layers = 1
learning_rate = 0.0024519997428423335
batch_size = 83
epochs = 99
lstm_units_layer_0 = 51
# best_error: 0.04012357832510543

In [ ]:
horizon = 10

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(LSTM(units=lstm_units_layer_0))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
LSTM_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
LSTM_MAE = mean_absolute_error(testY, testPredict)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("LSTM Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('LSTM Test RMSE : %.5f' % (LSTM_RMSE))
print('LSTM Test MAE : %.5f' % (LSTM_MAE))

Window Size:  10
17991 2000
(17833, 1, 149)
Epoch 1/99


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


215/215 [==============================] - 3s 2ms/step - loss: 0.0037
Epoch 2/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0021
Epoch 3/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0019
Epoch 4/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0018
Epoch 5/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0018
Epoch 6/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0017
Epoch 7/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0017
Epoch 8/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0017
Epoch 9/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 10/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 11/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 12/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0016
Epoch 13/99
215/215 [===============

### GRU Model

##### Horizon = 2

In [ ]:
# Best Params

look_back = 51
num_hidden_layers = 2
learning_rate = 0.001519335526892746
batch_size = 51
epochs = 82
lstm_units_layer_0 = 96
lstm_units_layer_1 = 208
# best_error: 0.020879777783223082

In [ ]:
horizon = 2

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(GRU(units=lstm_units_layer_0, return_sequences=True))
model.add(GRU(units=lstm_units_layer_1))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

# calculate root mean squared error
GRU_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
GRU_MAE = mean_absolute_error(testY, testPredict)

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("GRU Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('GRU Test RMSE : %.5f' % (GRU_RMSE))
print('GRU Test MAE : %.5f' % (GRU_MAE))

Window Size:  10
17991 2000
(17939, 1, 51)
Epoch 1/82


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


352/352 [==============================] - 5s 6ms/step - loss: 0.0022
Epoch 2/82
352/352 [==============================] - 3s 7ms/step - loss: 6.8747e-04
Epoch 3/82
352/352 [==============================] - 3s 10ms/step - loss: 6.3316e-04
Epoch 4/82
352/352 [==============================] - 3s 10ms/step - loss: 5.6578e-04
Epoch 5/82
352/352 [==============================] - 4s 11ms/step - loss: 5.8433e-04
Epoch 6/82
352/352 [==============================] - 4s 11ms/step - loss: 5.5823e-04
Epoch 7/82
352/352 [==============================] - 3s 10ms/step - loss: 5.4890e-04
Epoch 8/82
352/352 [==============================] - 4s 12ms/step - loss: 5.5914e-04
Epoch 9/82
352/352 [==============================] - 3s 9ms/step - loss: 5.3398e-04
Epoch 10/82
352/352 [==============================] - 3s 9ms/step - loss: 5.7062e-04
Epoch 11/82
352/352 [==============================] - 3s 7ms/step - loss: 5.3441e-04
Epoch 12/82
352/352 [==============================] - 3s 7ms/step - los

##### Horizon = 4

In [ ]:
# Best Params

# 0.02686025211281613
look_back = 56
num_hidden_layers = 3
learning_rate = 0.0056240033221932485
batch_size = 193
epochs = 78
lstm_units_layer_0 = 37
lstm_units_layer_1 = 82
lstm_units_layer_2 = 49

In [ ]:
horizon = 4

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(GRU(units=lstm_units_layer_0, return_sequences=True))
model.add(GRU(units=lstm_units_layer_1, return_sequences=True))
model.add(GRU(units=lstm_units_layer_2))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
GRU_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
GRU_MAE = mean_absolute_error(testY, testPredict)

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("GRU Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('GRU Test RMSE : %.5f' % (GRU_RMSE))
print('GRU Test MAE : %.5f' % (GRU_MAE))

Window Size:  10
17991 2000
(17932, 1, 56)
Epoch 1/78


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


93/93 [==============================] - 5s 6ms/step - loss: 0.0049
Epoch 2/78
93/93 [==============================] - 1s 6ms/step - loss: 0.0014
Epoch 3/78
93/93 [==============================] - 1s 6ms/step - loss: 0.0011
Epoch 4/78
93/93 [==============================] - 1s 7ms/step - loss: 0.0010
Epoch 5/78
93/93 [==============================] - 1s 9ms/step - loss: 9.6307e-04
Epoch 6/78
93/93 [==============================] - 1s 10ms/step - loss: 9.6427e-04
Epoch 7/78
93/93 [==============================] - 1s 7ms/step - loss: 9.1065e-04
Epoch 8/78
93/93 [==============================] - 1s 7ms/step - loss: 9.2314e-04
Epoch 9/78
93/93 [==============================] - 1s 7ms/step - loss: 8.7524e-04
Epoch 10/78
93/93 [==============================] - 1s 7ms/step - loss: 8.8676e-04
Epoch 11/78
93/93 [==============================] - 1s 6ms/step - loss: 8.6934e-04
Epoch 12/78
93/93 [==============================] - 1s 6ms/step - loss: 8.9678e-04
Epoch 13/78
93/93 [========

##### Horizon = 6

In [ ]:
# Best Params

look_back = 91
num_hidden_layers = 6
learning_rate = 0.0005124476844228014
batch_size = 50
epochs = 73
lstm_units_layer_0 = 50
lstm_units_layer_1 = 28
lstm_units_layer_2 = 82
lstm_units_layer_3 = 33
lstm_units_layer_4 = 22
lstm_units_layer_5 = 15
# best_error: 0.03235792738580053


In [ ]:
horizon = 6

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(GRU(units=lstm_units_layer_0, return_sequences=True))
model.add(GRU(units=lstm_units_layer_1, return_sequences=True))
model.add(GRU(units=lstm_units_layer_2, return_sequences=True))
model.add(GRU(units=lstm_units_layer_3, return_sequences=True))
model.add(GRU(units=lstm_units_layer_4, return_sequences=True))
model.add(GRU(units=lstm_units_layer_5))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
GRU_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
GRU_MAE = mean_absolute_error(testY, testPredict)

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("GRU Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('GRU Test RMSE : %.5f' % (GRU_RMSE))
print('GRU Test MAE : %.5f' % (GRU_MAE))

Window Size:  10
17991 2000
(17895, 1, 91)
Epoch 1/73


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


358/358 [==============================] - 12s 9ms/step - loss: 0.0091
Epoch 2/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0022
Epoch 3/73
358/358 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 4/73
358/358 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 5/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0013
Epoch 6/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0013
Epoch 7/73
358/358 [==============================] - 3s 8ms/step - loss: 0.0013
Epoch 8/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0012
Epoch 9/73
358/358 [==============================] - 4s 10ms/step - loss: 0.0012
Epoch 10/73
358/358 [==============================] - 3s 9ms/step - loss: 0.0012
Epoch 11/73
358/358 [==============================] - 3s 8ms/step - loss: 0.0012
Epoch 12/73
358/358 [==============================] - 3s 8ms/step - loss: 0.0011
Epoch 13/73
358/358 [=============

##### Horizon = 8

In [ ]:
# Best Params
look_back = 65
num_hidden_layers = 3
learning_rate = 0.00019511832310862206
batch_size = 8
epochs = 66
lstm_units_layer_0 = 8
lstm_units_layer_1 = 63
lstm_units_layer_2 = 240
# best_error: 0.036570963652281366

In [ ]:
horizon = 8

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(GRU(units=lstm_units_layer_0, return_sequences=True))
model.add(GRU(units=lstm_units_layer_1, return_sequences=True))
model.add(GRU(units=lstm_units_layer_2))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
GRU_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
GRU_MAE = mean_absolute_error(testY, testPredict)

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("GRU Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('GRU Test RMSE : %.5f' % (GRU_RMSE))
print('GRU Test MAE : %.5f' % (GRU_MAE))

Window Size:  10
17991 2000
(17919, 1, 65)
Epoch 1/66


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2240/2240 [==============================] - 17s 5ms/step - loss: 0.0039
Epoch 2/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0020
Epoch 3/66
2240/2240 [==============================] - 12s 6ms/step - loss: 0.0018
Epoch 4/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0016
Epoch 5/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0015
Epoch 6/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0015
Epoch 7/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0015
Epoch 8/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014
Epoch 9/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014
Epoch 10/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014
Epoch 11/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014
Epoch 12/66
2240/2240 [==============================] - 12s 5ms/step - loss: 0.0014


##### Horizon = 10

In [ ]:
# Best Params

look_back = 149
num_hidden_layers = 1
learning_rate = 0.0024519997428423335
batch_size = 83
epochs = 99
lstm_units_layer_0 = 51
# best_error: 0.04012357832510543

In [ ]:
horizon = 10

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(GRU(units=lstm_units_layer_0))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
GRU_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
GRU_MAE = mean_absolute_error(testY, testPredict)

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("GRU Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('GRU Test RMSE : %.5f' % (GRU_RMSE))
print('GRU Test MAE : %.5f' % (GRU_MAE))

Window Size:  10
17991 2000
(17833, 1, 149)
Epoch 1/99


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


215/215 [==============================] - 2s 3ms/step - loss: 0.0046
Epoch 2/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0022
Epoch 3/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0020
Epoch 4/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0019
Epoch 5/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0018
Epoch 6/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0018
Epoch 7/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0017
Epoch 8/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0017
Epoch 9/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0017
Epoch 10/99
215/215 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 11/99
215/215 [==============================] - 1s 4ms/step - loss: 0.0016
Epoch 12/99
215/215 [==============================] - 1s 4ms/step - loss: 0.0016
Epoch 13/99
215/215 [===============

### RNN Model

##### Horizon = 2

In [ ]:
# Best Params

look_back = 51
num_hidden_layers = 2
learning_rate = 0.001519335526892746
batch_size = 51
epochs = 82
lstm_units_layer_0 = 96
lstm_units_layer_1 = 208
# best_error: 0.020879777783223082

In [ ]:
horizon = 2

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(SimpleRNN(units=lstm_units_layer_0, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_1))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

# calculate root mean squared error
RNN_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
RNN_MAE = mean_absolute_error(testY, testPredict)

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("RNN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('RNN Test RMSE : %.5f' % (RNN_RMSE))
print('RNN Test MAE : %.5f' % (RNN_MAE))

Window Size:  10
17991 2000
(17939, 1, 51)
Epoch 1/82


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


352/352 [==============================] - 2s 3ms/step - loss: 0.0030
Epoch 2/82
352/352 [==============================] - 1s 3ms/step - loss: 6.6877e-04
Epoch 3/82
352/352 [==============================] - 1s 3ms/step - loss: 5.9263e-04
Epoch 4/82
352/352 [==============================] - 2s 4ms/step - loss: 5.9260e-04
Epoch 5/82
352/352 [==============================] - 1s 3ms/step - loss: 5.9019e-04
Epoch 6/82
352/352 [==============================] - 1s 3ms/step - loss: 5.3943e-04
Epoch 7/82
352/352 [==============================] - 1s 3ms/step - loss: 5.6368e-04
Epoch 8/82
352/352 [==============================] - 1s 3ms/step - loss: 5.5624e-04
Epoch 9/82
352/352 [==============================] - 1s 3ms/step - loss: 5.4106e-04
Epoch 10/82
352/352 [==============================] - 1s 3ms/step - loss: 5.7856e-04
Epoch 11/82
352/352 [==============================] - 1s 3ms/step - loss: 5.3231e-04
Epoch 12/82
352/352 [==============================] - 1s 3ms/step - loss: 5.5

##### Horizon = 4

In [ ]:
# Best Params

# 0.02686025211281613
look_back = 56
num_hidden_layers = 3
learning_rate = 0.0056240033221932485
batch_size = 193
epochs = 78
lstm_units_layer_0 = 37
lstm_units_layer_1 = 82
lstm_units_layer_2 = 49

In [ ]:
horizon = 4

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(SimpleRNN(units=lstm_units_layer_0, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_1, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_2))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
RNN_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
RNN_MAE = mean_absolute_error(testY, testPredict)

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("RNN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('RNN Test RMSE : %.5f' % (RNN_RMSE))
print('RNN Test MAE : %.5f' % (RNN_MAE))

Window Size:  10
17991 2000
(17932, 1, 56)
Epoch 1/78


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


93/93 [==============================] - 4s 6ms/step - loss: 0.0183
Epoch 2/78
93/93 [==============================] - 1s 7ms/step - loss: 0.0013
Epoch 3/78
93/93 [==============================] - 1s 6ms/step - loss: 0.0011
Epoch 4/78
93/93 [==============================] - 1s 5ms/step - loss: 9.7712e-04
Epoch 5/78
93/93 [==============================] - 1s 5ms/step - loss: 9.6193e-04
Epoch 6/78
93/93 [==============================] - 0s 3ms/step - loss: 9.3606e-04
Epoch 7/78
93/93 [==============================] - 1s 7ms/step - loss: 8.9329e-04
Epoch 8/78
93/93 [==============================] - 1s 5ms/step - loss: 9.4824e-04
Epoch 9/78
93/93 [==============================] - 1s 8ms/step - loss: 8.9808e-04
Epoch 10/78
93/93 [==============================] - 1s 8ms/step - loss: 8.6781e-04
Epoch 11/78
93/93 [==============================] - 1s 8ms/step - loss: 8.3249e-04
Epoch 12/78
93/93 [==============================] - 1s 8ms/step - loss: 8.5906e-04
Epoch 13/78
93/93 [=====

##### Horizon = 6

In [ ]:
# Best Params

look_back = 91
num_hidden_layers = 6
learning_rate = 0.0005124476844228014
batch_size = 50
epochs = 73
lstm_units_layer_0 = 50
lstm_units_layer_1 = 28
lstm_units_layer_2 = 82
lstm_units_layer_3 = 33
lstm_units_layer_4 = 22
lstm_units_layer_5 = 15
# best_error: 0.03235792738580053


In [ ]:
horizon = 6

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(SimpleRNN(units=lstm_units_layer_0, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_1, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_2, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_3, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_4, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_5))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
RNN_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
RNN_MAE = mean_absolute_error(testY, testPredict)

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("RNN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('RNN Test RMSE : %.5f' % (RNN_RMSE))
print('RNN Test MAE : %.5f' % (RNN_MAE))

Window Size:  10
17991 2000
(17895, 1, 91)
Epoch 1/73


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


358/358 [==============================] - 7s 5ms/step - loss: 0.0036
Epoch 2/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0018
Epoch 3/73
358/358 [==============================] - 2s 5ms/step - loss: 0.0016
Epoch 4/73
358/358 [==============================] - 2s 4ms/step - loss: 0.0014
Epoch 5/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0014
Epoch 6/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0013
Epoch 7/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0013
Epoch 8/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0012
Epoch 9/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0012
Epoch 10/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0012
Epoch 11/73
358/358 [==============================] - 2s 6ms/step - loss: 0.0012
Epoch 12/73
358/358 [==============================] - 1s 4ms/step - loss: 0.0012
Epoch 13/73
358/358 [===============

##### Horizon = 8

In [ ]:
# Best Params
look_back = 65
num_hidden_layers = 3
learning_rate = 0.00019511832310862206
batch_size = 8
epochs = 66
lstm_units_layer_0 = 8
lstm_units_layer_1 = 63
lstm_units_layer_2 = 240
# best_error: 0.036570963652281366

In [ ]:
horizon = 8

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(SimpleRNN(units=lstm_units_layer_0, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_1, return_sequences=True))
model.add(SimpleRNN(units=lstm_units_layer_2))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
RNN_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
RNN_MAE = mean_absolute_error(testY, testPredict)

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("RNN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('RNN Test RMSE : %.5f' % (RNN_RMSE))
print('RNN Test MAE : %.5f' % (RNN_MAE))

Window Size:  10
17991 2000
(17919, 1, 65)
Epoch 1/66


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2240/2240 [==============================] - 8s 3ms/step - loss: 0.0024
Epoch 2/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0017
Epoch 3/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0016
Epoch 4/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0015
Epoch 5/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0015
Epoch 6/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0015
Epoch 7/66
2240/2240 [==============================] - 7s 3ms/step - loss: 0.0014
Epoch 8/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0014
Epoch 9/66
2240/2240 [==============================] - 7s 3ms/step - loss: 0.0014
Epoch 10/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0014
Epoch 11/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0014
Epoch 12/66
2240/2240 [==============================] - 6s 3ms/step - loss: 0.0014
Epoch 13/66


##### Horizon = 10

In [ ]:
# Best Params

look_back = 149
num_hidden_layers = 1
learning_rate = 0.0024519997428423335
batch_size = 83
epochs = 99
lstm_units_layer_0 = 51
# best_error: 0.04012357832510543

In [ ]:
horizon = 10

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape)

# Crée et entraîne le modèle pour l'horizon de prévision i
model = Sequential()
model.add(SimpleRNN(units=lstm_units_layer_0))
model.add(Dense(horizon))
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Record the starting time to training the model
training_start_time = time.time()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=1)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time


# Record the starting time to generate predictions
predictions_start_time = time.time()

testPredict = model.predict(testX, verbose=0)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time
# testPredict = np.reshape(testPredict, (testPredict.shape[0], testPredict.shape[2]))
# calculate root mean squared error
RNN_RMSE = np.sqrt(mean_squared_error(testY, testPredict))
RNN_MAE = mean_absolute_error(testY, testPredict)

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("RNN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('RNN Test RMSE : %.5f' % (RNN_RMSE))
print('RNN Test MAE : %.5f' % (RNN_MAE))

Window Size:  10
17991 2000
(17833, 1, 149)
Epoch 1/99


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


215/215 [==============================] - 1s 3ms/step - loss: 0.0085
Epoch 2/99
215/215 [==============================] - 1s 2ms/step - loss: 0.0026
Epoch 3/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 4/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 5/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 6/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 7/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 8/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 9/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 10/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 11/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 12/99
215/215 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 13/99
215/215 [===============

##### ESN Model

##### Finding Best Hyperparameters

In [ ]:
horizon = 2
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], validX.shape[1]))

    model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(validX)
    testPredict = np.array(testPredict)

    testRMSE = np.sqrt(mean_squared_error(validY, testPredict[:len(validY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-05 18:18:35,699] A new study created in memory with name: no-name-ddc9082b-727e-4ba8-a494-5efd51d9f595


Window Size:  10
17991 16192 1799 2000
harvesting states...
fitting...
training error:
0.010833627462823557


[I 2023-09-05 18:18:44,487] Trial 0 finished with value: 0.021963175200418433 and parameters: {'n_reservoir': 776, 'sparsity': 0.2, 'spectral_radius': 0.9, 'noise': 0.004, 'look_back': 129}. Best is trial 0 with value: 0.021963175200418433.


harvesting states...
fitting...
training error:
0.011537171838086738


[I 2023-09-05 18:18:54,159] Trial 1 finished with value: 0.022200259535445747 and parameters: {'n_reservoir': 808, 'sparsity': 0.4, 'spectral_radius': 0.9, 'noise': 0.007, 'look_back': 36}. Best is trial 0 with value: 0.021963175200418433.


...

training error:
0.018674490297960523
harvesting states...
fitting...


[I 2023-09-05 18:23:44,205] Trial 98 finished with value: 0.02065796669612801 and parameters: {'n_reservoir': 123, 'sparsity': 0.5, 'spectral_radius': 0.8, 'noise': 0.007, 'look_back': 274}. Best is trial 63 with value: 0.019390472901838596.


training error:
0.011174924886488273
harvesting states...
fitting...


[I 2023-09-05 18:23:45,846] Trial 99 finished with value: 0.019474154392976388 and parameters: {'n_reservoir': 31, 'sparsity': 0.3, 'spectral_radius': 10.0, 'noise': 0.006, 'look_back': 307}. Best is trial 63 with value: 0.019390472901838596.


training error:
0.018723914456959683
done
ESN HyperParameters Tuning Elapsed Time : 310.14912 seconds
Best Parameters:  {'n_reservoir': 13, 'sparsity': 0.01, 'spectral_radius': 10.0, 'noise': 0.006, 'look_back': 328}
Best Loss:  0.019390472901838596


In [ ]:
horizon = 4
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], validX.shape[1]))

    model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(validX)
    testPredict = np.array(testPredict)

    testRMSE = np.sqrt(mean_squared_error(validY, testPredict[:len(validY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-05 18:23:45,915] A new study created in memory with name: no-name-6c83b9dd-b020-45b5-b1dd-768d0201cefb


Window Size:  10
17991 16192 1799 2000
harvesting states...
fitting...


[I 2023-09-05 18:23:49,994] Trial 0 finished with value: 0.026487567172234574 and parameters: {'n_reservoir': 460, 'sparsity': 0.3, 'spectral_radius': 0.2, 'noise': 0.007, 'look_back': 121}. Best is trial 0 with value: 0.026487567172234574.


training error:
0.007830198652219306
harvesting states...
fitting...
training error:
0.007796594803344706


[I 2023-09-05 18:23:54,817] Trial 1 finished with value: 0.027518801577407607 and parameters: {'n_reservoir': 526, 'sparsity': 0.4, 'spectral_radius': 0.5, 'noise': 0.005, 'look_back': 186}. Best is trial 0 with value: 0.026487567172234574.


...

training error:
0.0240487265844705
harvesting states...
fitting...


[I 2023-09-05 18:28:14,912] Trial 98 finished with value: 0.030190745673855608 and parameters: {'n_reservoir': 120, 'sparsity': 0.01, 'spectral_radius': 0.3, 'noise': 0.001, 'look_back': 358}. Best is trial 90 with value: 0.0251564879356312.


training error:
0.008996800023784862
harvesting states...
fitting...


[I 2023-09-05 18:28:17,093] Trial 99 finished with value: 0.025426904421104853 and parameters: {'n_reservoir': 31, 'sparsity': 0.01, 'spectral_radius': 10.0, 'noise': 0.001, 'look_back': 400}. Best is trial 90 with value: 0.0251564879356312.


training error:
0.023971836243246756
done
ESN HyperParameters Tuning Elapsed Time : 271.18417 seconds
Best Parameters:  {'n_reservoir': 11, 'sparsity': 0.01, 'spectral_radius': 10.0, 'noise': 0.001, 'look_back': 340}
Best Loss:  0.0251564879356312


In [ ]:
horizon = 6
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], validX.shape[1]))

    model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(validX)
    testPredict = np.array(testPredict)

    testRMSE = np.sqrt(mean_squared_error(validY, testPredict[:len(validY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-05 18:28:17,152] A new study created in memory with name: no-name-0d7b7fb5-6452-4bc7-832c-c98c28af3513


Window Size:  10
17991 16192 1799 2000
harvesting states...
fitting...


[I 2023-09-05 18:28:21,271] Trial 0 finished with value: 0.04166467927237724 and parameters: {'n_reservoir': 290, 'sparsity': 0.4, 'spectral_radius': 1, 'noise': 0.002, 'look_back': 435}. Best is trial 0 with value: 0.04166467927237724.


training error:
0.008251314010583358
harvesting states...
fitting...
training error:
0.006480920304544917


[I 2023-09-05 18:28:35,083] Trial 1 finished with value: 0.03551645308697249 and parameters: {'n_reservoir': 939, 'sparsity': 0.1, 'spectral_radius': 0.4, 'noise': 0.004, 'look_back': 374}. Best is trial 1 with value: 0.03551645308697249.


...

training error:
0.007130952739219939
harvesting states...
fitting...
training error:
0.006626008770791709


[I 2023-09-05 18:34:46,822] Trial 98 finished with value: 0.03615291426368435 and parameters: {'n_reservoir': 301, 'sparsity': 0.5, 'spectral_radius': 0.3, 'noise': 0.004, 'look_back': 68}. Best is trial 2 with value: 0.030801385450078596.


harvesting states...
fitting...


[I 2023-09-05 18:34:49,695] Trial 99 finished with value: 0.03183808416693582 and parameters: {'n_reservoir': 342, 'sparsity': 0.5, 'spectral_radius': 0.3, 'noise': 0.004, 'look_back': 88}. Best is trial 2 with value: 0.030801385450078596.


training error:
0.006654324077371295
done
ESN HyperParameters Tuning Elapsed Time : 392.54067 seconds
Best Parameters:  {'n_reservoir': 485, 'sparsity': 0.3, 'spectral_radius': 0.5, 'noise': 0.001, 'look_back': 71}
Best Loss:  0.030801385450078596


In [ ]:
horizon = 8
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], validX.shape[1]))

    model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(validX)
    testPredict = np.array(testPredict)

    testRMSE = np.sqrt(mean_squared_error(validY, testPredict[:len(validY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-05 18:34:49,735] A new study created in memory with name: no-name-179a7dd7-1741-42aa-8146-9e9788c9040a


Window Size:  10
17991 16192 1799 2000
harvesting states...
fitting...
training error:
0.007241253937770114


[I 2023-09-05 18:34:56,266] Trial 0 finished with value: 0.04310083943641763 and parameters: {'n_reservoir': 464, 'sparsity': 0.2, 'spectral_radius': 1.1, 'noise': 0.005, 'look_back': 493}. Best is trial 0 with value: 0.04310083943641763.


harvesting states...
fitting...
training error:
0.006026653454832251


[I 2023-09-05 18:35:05,474] Trial 1 finished with value: 0.03935861684859025 and parameters: {'n_reservoir': 721, 'sparsity': 0.3, 'spectral_radius': 1.1, 'noise': 0.004, 'look_back': 289}. Best is trial 1 with value: 0.03935861684859025.


...

training error:
0.00619563463917762
harvesting states...
fitting...
training error:
0.005936637576383639


[I 2023-09-05 18:47:53,948] Trial 98 finished with value: 0.03575908481061348 and parameters: {'n_reservoir': 493, 'sparsity': 0.2, 'spectral_radius': 0.5, 'noise': 0.005, 'look_back': 29}. Best is trial 80 with value: 0.035200750018335836.


harvesting states...
fitting...
training error:
0.006382864695717489


[I 2023-09-05 18:47:57,224] Trial 99 finished with value: 0.037455476547877724 and parameters: {'n_reservoir': 495, 'sparsity': 0.2, 'spectral_radius': 0.5, 'noise': 0.005, 'look_back': 11}. Best is trial 80 with value: 0.035200750018335836.


done
ESN HyperParameters Tuning Elapsed Time : 787.49570 seconds
Best Parameters:  {'n_reservoir': 981, 'sparsity': 0.1, 'spectral_radius': 1, 'noise': 0.003, 'look_back': 39}
Best Loss:  0.035200750018335836


In [ ]:
horizon = 10
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
valid_size = int(len(train) * 0.1)
temp_train = train[:-valid_size]
valid = train[-valid_size:]
print(len(train), len(temp_train), len(valid), len(test))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1]))
    validX = np.reshape(validX, (validX.shape[0], validX.shape[1]))

    model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(validX)
    testPredict = np.array(testPredict)

    testRMSE = np.sqrt(mean_squared_error(validY, testPredict[:len(validY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-05 18:47:57,261] A new study created in memory with name: no-name-1bf0bcb8-87f6-4085-aecd-399fd0c54daa


Window Size:  10
17991 16192 1799 2000
harvesting states...
fitting...


[I 2023-09-05 18:47:58,983] Trial 0 finished with value: 0.0457867247462088 and parameters: {'n_reservoir': 225, 'sparsity': 0.4, 'spectral_radius': 1.25, 'noise': 0.001, 'look_back': 115}. Best is trial 0 with value: 0.0457867247462088.


training error:
0.00694723702183422
harvesting states...
fitting...


[I 2023-09-05 18:48:01,976] Trial 1 finished with value: 0.6130436169988811 and parameters: {'n_reservoir': 167, 'sparsity': 0.3, 'spectral_radius': 0.3, 'noise': 0.008, 'look_back': 467}. Best is trial 0 with value: 0.0457867247462088.


...

harvesting states...
fitting...
training error:
0.005333974368127683


[I 2023-09-05 18:55:40,363] Trial 98 finished with value: 0.040548108578194304 and parameters: {'n_reservoir': 511, 'sparsity': 0.1, 'spectral_radius': 0.3, 'noise': 0.001, 'look_back': 56}. Best is trial 92 with value: 0.03893053466437693.


harvesting states...
fitting...


[I 2023-09-05 18:55:43,783] Trial 99 finished with value: 0.04060653331963552 and parameters: {'n_reservoir': 460, 'sparsity': 0.1, 'spectral_radius': 0.7, 'noise': 0.001, 'look_back': 96}. Best is trial 92 with value: 0.03893053466437693.


training error:
0.005566443565576262
done
ESN HyperParameters Tuning Elapsed Time : 466.52411 seconds
Best Parameters:  {'n_reservoir': 579, 'sparsity': 0.1, 'spectral_radius': 0.3, 'noise': 0.001, 'look_back': 52}
Best Loss:  0.03893053466437693


##### Horizon = 2

In [ ]:
# Best Params
n_reservoir = 13
sparsity = 0.01
spectral_radius = 10.0
noise = 0.006
look_back = 328

In [ ]:
horizon = 2

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# Build and fit the ESN model

model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
            )

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

predictions = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

predictions = np.array(predictions)

# calculate root mean squared error
ESN_RMSE = np.sqrt(mean_squared_error(testY, predictions))
ESN_MAE = mean_absolute_error(testY, predictions)

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("ESN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('ESN Test RMSE : %.5f' % (ESN_RMSE))
print('ESN Test MAE : %.5f' % (ESN_MAE))

Window Size:  10
17991 2000
harvesting states...
fitting...
training error:
0.018728522009361006
ESN Model Training Elapsed Time : 1.74218 seconds
ESN Model Generate predictions Elapsed Time : 0.07664 seconds
ESN Test RMSE : 0.01939
ESN Test MAE : 0.01596


##### Horizon = 4

In [ ]:
n_reservoir = 11
sparsity = 0.01
spectral_radius = 10.0
noise = 0.001
look_back = 340

In [ ]:
horizon = 4

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# Build and fit the ESN model

model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
            )

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

predictions = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

predictions = np.array(predictions)

# calculate root mean squared error
ESN_RMSE = np.sqrt(mean_squared_error(testY, predictions))
ESN_MAE = mean_absolute_error(testY, predictions)

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("ESN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('ESN Test RMSE : %.5f' % (ESN_RMSE))
print('ESN Test MAE : %.5f' % (ESN_MAE))

Window Size:  10
17991 2000
harvesting states...
fitting...
training error:
0.02414602751770763
ESN Model Training Elapsed Time : 1.71317 seconds
ESN Model Generate predictions Elapsed Time : 0.09333 seconds
ESN Test RMSE : 0.02516
ESN Test MAE : 0.01997


##### Horizon = 6

In [ ]:
n_reservoir = 485
sparsity = 0.3
spectral_radius = 0.5
noise = 0.001
look_back = 71


In [ ]:
horizon = 6

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# Build and fit the ESN model

model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
            )

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

predictions = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

predictions = np.array(predictions)

# calculate root mean squared error
ESN_RMSE = np.sqrt(mean_squared_error(testY, predictions))
ESN_MAE = mean_absolute_error(testY, predictions)

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("ESN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('ESN Test RMSE : %.5f' % (ESN_RMSE))
print('ESN Test MAE : %.5f' % (ESN_MAE))

Window Size:  10
17991 2000
harvesting states...
fitting...
training error:
0.006551621192409506
ESN Model Training Elapsed Time : 7.33919 seconds
ESN Model Generate predictions Elapsed Time : 0.38094 seconds
ESN Test RMSE : 0.03080
ESN Test MAE : 0.02410


##### Horizon = 8

In [ ]:
n_reservoir = 981
sparsity = 0.1
spectral_radius = 1
noise = 0.003
look_back = 39


In [ ]:
horizon = 8

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# Build and fit the ESN model

model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
            )

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

predictions = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

predictions = np.array(predictions)

# calculate root mean squared error
ESN_RMSE = np.sqrt(mean_squared_error(testY, predictions))
ESN_MAE = mean_absolute_error(testY, predictions)

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("ESN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('ESN Test RMSE : %.5f' % (ESN_RMSE))
print('ESN Test MAE : %.5f' % (ESN_MAE))

Window Size:  10
17991 2000
harvesting states...
fitting...
training error:
0.005699949050913774
ESN Model Training Elapsed Time : 16.30125 seconds
ESN Model Generate predictions Elapsed Time : 1.09223 seconds
ESN Test RMSE : 0.03520
ESN Test MAE : 0.02723


##### Horizon = 10

In [ ]:
n_reservoir = 579
sparsity = 0.1
spectral_radius = 0.3
noise = 0.001
look_back = 52

In [ ]:
horizon = 10

node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# Build and fit the ESN model

model = ESN(n_inputs = look_back,
            n_outputs = horizon,
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
            teacher_scaling = 10,
            )

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

predictions = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

predictions = np.array(predictions)

# calculate root mean squared error
ESN_RMSE = np.sqrt(mean_squared_error(testY, predictions))
ESN_MAE = mean_absolute_error(testY, predictions)

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")
print("ESN Model Generate predictions Elapsed Time : %.5f" % (predictions_elapsed_time), "seconds")
print('ESN Test RMSE : %.5f' % (ESN_RMSE))
print('ESN Test MAE : %.5f' % (ESN_MAE))

Window Size:  10
17991 2000
harvesting states...
fitting...
training error:
0.0052852026198604764
ESN Model Training Elapsed Time : 7.05726 seconds
ESN Model Generate predictions Elapsed Time : 0.42938 seconds
ESN Test RMSE : 0.03893
ESN Test MAE : 0.03022
